## Importing needed packages

In [202]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

## Defining Input Data

In [203]:
# Create a dataframe for Capacity Cost
CapCost = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'CapCost[€/MW]': [150, 100, 2500, 300, 10]
})

# Create a dataframe for Operating Cost
OpCost = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'OpCost[€/MWh]': [200, 250, 50, 0, 0]
})

# Create a dataframe to define general technology information
TechInfo = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'Type': ['Fossil', 'Fossil', 'Fossil', 'RES', 'RES']
})

# Create a dataframe for Storage Cost
StorCost = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'StorCost[€/MWh]': [10, 50000000]
})

# Create a dataframe for Capacity Limit
CapLim = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'CapLim[MW]': [100, 100, 100, 10000, 10000]
})

# Create a dataframe for existing capacity
CapExi = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'ExCap[MW]': [100, 50, 50, 100, 100]
})

# Create a dataframe for outphased capacity
CapOut = pd.DataFrame({
    'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
    'OutCap[MW]': [0, 0, 0, -50, 0]
})
# Create a dataframe for production factor on hourly basis
ProdFac = pd.DataFrame({
    'Coal': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'Gas': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'Nuclear': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'Wind': [0.07149117259925315, 0.20690652911053053, 0.17089549294945167, 0.18881997277064186, 
    0.20005366722911527, 0.36282711876056234, 0.4851625621195393, 0.609154052662755, 
    0.6635074058672907, 0.8744552242877893, 0.7799802294571636, 0.8414978281203519, 
    0.9364414827427749, 0.9645481320075096, 0.9312765663817734, 0.7834986690899264, 
    0.6829013245988148, 0.5967793277038156, 0.4073780665068157, 0.33104623367164343, 
    0.33071882539854985, 0.24498913233631167, 0.1011445772753432, 0.13822843349204414],
    'Solar': [0.003151111598444441, 0.007907054051593435, 0.01831563888873418, 0.039163895098987066, 
    0.07730474044329971, 0.14085842092104503, 0.23692775868212176, 0.3678794411714424, 
    0.5272924240430484, 0.697676326071031, 0.8521437889662112, 0.9607894391523233, 
    1.0, 0.9607894391523233, 0.8521437889662112, 0.697676326071031, 
    0.5272924240430484, 0.3678794411714424, 0.23692775868212176, 0.14085842092104503, 
    0.07730474044329971, 0.039163895098987066, 0.01831563888873418, 0.007907054051593435]
})

# Create a dataframe for demand on hourly basis
Demand = pd.DataFrame({
    'Hour': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    'Demand[MWh]': [500,600,555,343,644,343,535,223,535,634,343,535,223,535,634,343,535,223,535,634,343,535,223,535]
})

# Create a dataframe for eta charge
EtaCh = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'EtaCh': [0.9, 0.8]
})

# Create a dataframe for eta discharge
EtaDis = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'EtaDis': [0.9, 0.8]
})

# Create a dataframe for existing storage capacity
StorExi = pd.DataFrame({
    'Plant': ['Battery', 'Pumped Hydro'],
    'ExStor[MWh]': [40, 0]
})


## Defining Input Parameters

In [204]:
# Define number of hours as length of demand
N_Hours = len(Demand)

# Define number of generators as length of CapCost
N_Cap = len(CapCost)

# Define number of storage as length of StorCost
N_Stor = len(StorCost)

# Define epsilon
epsilon = 0.9

#Define delta
delta = 0.8

## Defining classes

In [205]:
# Class for external input data
class InputData():
    def __init__(self, CapCost, OpCost, TechInfo, StorCost, CapLim,CapExi,CapOut,ProdFactor,Dem,EtaCha,EtaDis,StorExi):
        self.CapCost = CapCost
        self.OpCost = OpCost
        self.TechInfo = TechInfo
        self.StorCost = StorCost
        self.CapLim = CapLim
        self.CapExi = CapExi
        self.CapOut = CapOut
        self.ProdFactor = ProdFactor
        self.Dem = Dem
        self.EtaCha = EtaCha
        self.EtaDis = EtaDis
        self.StorExi = StorExi

        

In [206]:
# Class for model parameters
class Parameters():
    def __init__(self, epsilon, delta, N_Hours, N_Cap, N_Stor):
        self.epsilon = epsilon
        self.delta = delta
        self.N_Hours = N_Hours
        self.N_Cap = N_Cap
        self.N_Stor = N_Stor

## Creating Data and Parameter Objects

In [207]:
ParametersObj = Parameters(epsilon, delta, N_Hours, N_Cap, N_Stor)
DataObj = InputData(CapCost, OpCost, TechInfo, StorCost, CapLim,CapExi,CapOut,ProdFac,Demand,EtaCh,EtaDis,StorExi)


In [208]:
DataObj.TechInfo['Type'][0]

'Fossil'

## Creating the Model

In [209]:
# CLASS WHICH CAN HAVE ATTRIBUTES SET

class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [210]:
# Defining the optimization model class

class CapacityProblem():
    def __init__(self, ParametersObj, DataObj, Model_results = 1, Guroby_results = 1):
        self.P = ParametersObj # Parameters
        self.D = DataObj # Data
        self.Model_results = Model_results
        self.Guroby_results = Guroby_results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 


    def _build_variables(self):
        # Create the variables
        self.var.CapNew = self.m.addMVar((self.P.N_Cap), lb=0)  # New Capacity for each type of generator technology
        self.var.EGen = self.m.addMVar((self.P.N_Cap, self.P.N_Hours), lb=0)  # Energy Production for each type of generator technology for each hour and scenario
        self.var.CapStor = self.m.addMVar((self.P.N_Stor), lb=0)  # New storage capacity for each type of storage technology
        self.var.SOC = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # State of charge for each type of storage technology for each hour and scenario  
        self.var.EChar = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # Energy charged for each type of storage technology for each hour and scenario
        self.var.EDis = self.m.addMVar((self.P.N_Stor, self.P.N_Hours), lb=0)  # Energy discharged for each type of storage technology for each hour and scenario


    def _build_constraints(self):
        # Limit new capacity by maximum investable capacity for each type of generator technology
        for g in range(self.P.N_Cap):
            self.con.CapLim = self.m.addConstr(self.var.CapNew[g] <= self.D.CapLim['CapLim[MW]'][g], name=f'Capacity limit_{g}')
        
        # Production limited by sum of new capacity, existing capacity and phased out capacity times the production factor
        for g in range(self.P.N_Cap):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.ProdLim = self.m.addConstr(self.var.EGen[g, h] <= (self.var.CapNew[g] + self.D.CapExi['ExCap[MW]'][g] + self.D.CapOut['OutCap[MW]'][g]) * self.D.ProdFactor.iloc[h,g], name='Production limit')

        # Energy Balance constraint - generation and discharge needs to equal demand and charging for each hour and scenario
        for h in range(self.P.N_Hours):
            #for s in range(self.P.N_Scen):
                self.con.Balance = self.m.addConstr(gp.quicksum(self.var.EGen[:, h]) + gp.quicksum(self.var.EDis[:, h]) == self.D.Dem['Demand[MWh]'][h] + gp.quicksum(self.var.EChar[:, h]), name='Energy balance')

        # Defining RES share as a percentage of total energy demand
        self.con.RESShare = self.m.addConstr(gp.quicksum(self.var.EGen[g, h] for h in range(self.P.N_Hours) for g in range(self.P.N_Cap) if self.D.TechInfo['Type'][g] == 'RES' ) >= self.P.delta * gp.quicksum(self.D.Dem['Demand[MWh]'][h] for h in range(self.P.N_Hours)), name='RES share')

        # Defining SOC as SOC from previous hour plus energy charged minus energy discharged for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(1, self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.SOC = self.m.addConstr(self.var.SOC[u, h] == self.var.SOC[u, h-1] + self.var.EChar[u, h-1] - self.var.EDis[u, h-1], name='State of charge')

        # Define SOC for first hour
        for u in range(self.P.N_Stor):
            #for s in range(self.P.N_Scen):
                self.con.SOC0 = self.m.addConstr(self.var.SOC[u, 0] == (self.D.StorExi['ExStor[MWh]'][u])*0, name='State of charge 0')

        # Limit SOC by maximum storage capacity for each type of storage technology
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.SOCLim = self.m.addConstr(self.var.SOC[u, h] <= self.var.CapStor[u] + self.D.StorExi['ExStor[MWh]'][u], name='Storage capacity limit')

        # Limit charged energy by capacity minus state of charge for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.ECharLim = self.m.addConstr(self.var.EChar[u, h] <= self.var.CapStor[u] + self.D.StorExi['ExStor[MWh]'][u] - self.var.SOC[u, h], name='Energy charged limit')

        # Limit discharged energy by state of charge for each hour and scenario
        for u in range(self.P.N_Stor):
            for h in range(self.P.N_Hours):
                #for s in range(self.P.N_Scen):
                    self.con.EDisLim = self.m.addConstr(self.var.EDis[u, h] <= self.var.SOC[u, h], name='Energy discharged limit')


        
    
    def _build_objective(self):
        # Objective function
        objective = (gp.quicksum(self.var.CapNew[g] * self.D.CapCost['CapCost[€/MW]'][g] for g in range(self.P.N_Cap))  # Cost for new capacity for each type of generator technology
        + gp.quicksum(self.var.EGen[g,h] * self.D.OpCost['OpCost[€/MWh]'][g] for g in range(self.P.N_Cap) for h in range(self.P.N_Hours)) # Operating cost for each type of generator technology for each hour and scenario
        + gp.quicksum(self.var.CapStor[u] * self.D.StorCost['StorCost[€/MWh]'][u] for u in range(self.P.N_Stor))) # Cost for new storage capacity for each type of storage technology
        self.m.setObjective(objective, GRB.MINIMIZE)


    def _display_guropby_results(self):
        self.m.setParam('OutputFlag', self.Guroby_results)

    
    def _build_model(self):
        self.m = gp.Model('CapacityProblem')
        self._build_variables()
        self._build_constraints()
        self._build_objective()
        self._display_guropby_results()
        self.m.write("CapacityProblem.lp")
        self.m.optimize()
        self._results()
        if self.Model_results == 1:
            self._extract_results()
            

    
    def _results(self):
        self.res.obj = self.m.objVal
        self.res.CapNew = self.var.CapNew.X
        self.res.EGen = self.var.EGen.X
        self.res.CapStor = self.var.CapStor.X
        self.res.SOC = self.var.SOC.X
        self.res.EChar = self.var.EChar.X
        self.res.EDis = self.var.EDis.X
         
        
        #Create Dataframe for new capacity
        self.res.CapNew = pd.DataFrame({
            'Plant': ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar'],
            'CapNew[MW]': self.res.CapNew
        })

        #Create Dataframe for new storage capacity
        self.res.CapStor = pd.DataFrame({
            'Plant': ['Battery', 'Pumped Hydro'],
            'CapStor[MWh]': self.res.CapStor
        })

        #Create one Dataframe for demand, energy production, state of charge, energy charged and energy discharged
        self.res.Demand = self.D.Dem
        self.res.EGen = pd.DataFrame(self.res.EGen.T, columns = ['Coal Gen', 'Gas Gen', 'Nuclear Gen', 'Wind Gen', 'Solar Gen'])  
        self.res.SOC = pd.DataFrame(self.res.SOC.T, columns = ['Battery SOC', 'Pumped Hydro SOC'])
        self.res.EChar = pd.DataFrame(self.res.EChar.T, columns = ['Battery Char', 'Pumped Hydro Char'])
        self.res.EDis = pd.DataFrame(self.res.EDis.T, columns = ['Battery Dis', 'Pumped Hydro Dis'])

        #Combine all dataframes into one
        self.res.All = pd.concat([self.res.Demand, self.res.EGen, self.res.SOC, self.res.EChar, self.res.EDis], axis=1)
        
        

        
    

    def _extract_results(self):
        # Display the objective value
        print('Objective value: ', self.m.objVal)
        # Display the new capacity for each type of generator technology
        print('New capacity for each type of generator technology: ', self.res.CapNew)
        
        #Display new storage capacity for each type of storage technology
        print('New storage capacity for each type of storage technology: ', self.res.CapStor)

        

## Execute the model

In [211]:
CapacityProblem = CapacityProblem(ParametersObj, DataObj)
#CapacityProblem._results()
#CapacityProblem._extract_results()

ResultsCapacity = CapacityProblem.res.CapNew
ResultsStorage = CapacityProblem.res.CapStor
AllResults = CapacityProblem.res.All

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 342 rows, 271 columns and 1031 nonzeros
Model fingerprint: 0xe8ec8543
Coefficient statistics:
  Matrix range     [3e-03, 1e+00]
  Objective range  [1e+01, 5e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+04]
Presolve removed 49 rows and 40 columns
Presolve time: 0.00s
Presolved: 293 rows, 231 columns, 1104 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.780591e+03   0.000000e+00      0s
     116    5.4307323e+05   0.000000e+00   0.000000e+00      0s

Solved in 116 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.430732301e+05
Objective value:  543073.2300550715
New capacity for each type of generator technology:       Plant    C